In [1]:
import asyncio
import nest_asyncio
import os
import sys

from dotenv import load_dotenv
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [2]:
nest_asyncio.apply()

In [3]:
urls=[
    "https://www.spirit.com/",
    "https://www.spirit.com/s/about_us",
    # "https://foundation.spirit.com/es-US/foundation/?_gl=1*1y7b1nj*_gcl_au*MTg3NTczMzE1NC4xNzM3MDQxNzkw*_ga*NjQ4NzA5MjgzLjE3MzcwNDE3OTA.*_ga_VKEG2ZBNZ5*MTczNzE0NjU0My40LjAuMTczNzE0NjU0NC41OS4wLjMzMTI1NzM3MQ..",
    # "https://www.spirit.com/free-spirit",
    # "https://www.spirit.com/savers-club",
    # "https://www.spirit.com/press-release"
    ]

In [4]:
async def load_and_process_documents():
    loader = AsyncChromiumLoader(urls)
    tt = Html2TextTransformer()

    docs = tt.transform_documents(loader.load())
    print(docs)
    
    # Dividir los documentos en chunks más pequeños
    ts = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    fd = ts.split_documents(docs)
    
    return fd

In [5]:
fd = await load_and_process_documents()
print(len(fd))
l = []
client = OpenAI()

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://www.spirit.com/'}, page_content='Sign-In\n\n  * # >> Since \n\n#  >> Since\n\n  * 0 PTS\n  * My Dashboard\n  * Edit profile\n  * Free spirit Status\n  * Activity\n  * Saver$ Club\n  * Sign Out\n\nEspañol __\n\n  * Hi,\n  * # >> Since \n\n#  >> Since\n\n  * 0 PTS\n  * My Dashboard\n  * Edit profile\n  * Free spirit Status\n  * Activity\n  * Saver$ Club\n  * Sign Out\n\nEspañol __\n\n  * Book\n  * * * *\n\nMy Trips\n\n  * * * *\n\nCheck In\n\n  * * * *\n\nFlight Status\n\n  * * * *\n\nTravel Info\n\n __\n\n  * * * *\n\nLoyalty\n\n __\n\n  * * * *\n\nDeals\n\n __\n\n  * * * *\n\nContact Us\n\n  * * * *\n\n __Español\n\n  * Book\n  * My Trips\n  * Check In\n  * Flight Status\n  * Travel Info \n\nOnboard Experience\n\nWi-fi\n\n  * Loyalty \n\nFree Spirit®\n\nSaver$ Club\n\n  * Deals \n\nFlight Deals\n\nVacation Deals\n\n  * Contact Us \n  * Sign-In __\n\n  * ES __\n\nSlide 1 of 1\n\nTravel Advisory: Los Angeles Area Wildfires\n\n  * FLIGHT \n\n  * BUND

In [32]:
for xx in fd:
    response = completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are a highly intelligent assistant capable of breaking text into structured summaries."
        },
        {
            "role": "user",
            "content": (
                "Break down the following text into structured bullet points using the format: "
                "'topic | subtopic | content'.\n\n"
                "Guidelines:\n"
                "- Focus only on meaningful sentences.\n"
                "- Ignore headings, filler words, or unimportant details.\n"
                "- Use concise yet descriptive sentences for the 'content'.\n\n"
                "Text to summarize:\n\n"
                f"{xx}"
            )
        }]
    )
    l.append(response.choices[0].message.content)

print("".join(l))

Here is a structured breakdown of the provided text:

- Sign-In | Options | Access options like dashboard, edit profile, and sign out.
- Programs | Free Spirit Status | Information regarding loyalty program status.
- Programs | Saver$ Club | Membership program details.
- Bookings | Options | Options for booking flights, hotels, cars, and cruises.
- Flights | Booking Types | Options include Round Trip, One Way, Multi-City.
- Passengers | Adults | Ages 18 and up.
- Passengers | Children | Ages 0 to 17.
- Special Offers | Promo Code | Option to apply a promo code for discounts.
- Flight Search | Points | Show fares in points.
- Travel Information | Depart & Return | Dates selection for travel.
- Travel Information | Bundle & Save | Option to add a hotel or car to a flight booking.
- Optional Services | Bag Prices | Information on new bag prices.
- Travel Styles | Go Big, Go Comfy, Go Savvy | Four distinct travel options available.
- Fees | No Change or Cancel Fees | Policy on change or ca

: 

: 

: 

In [21]:
import json

formatted_list = [item.replace("\n", "\n") for item in l]

with open("output.json", "w", encoding="utf-8") as f:
    json.dump(formatted_list, f, ensure_ascii=False, indent=4)

print("Results saved to output.json")

Results saved to output.json


: 

: 

: 